In [22]:

import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np

df = pd.read_csv('data/processed/node-first-accident.csv')
df['accidentdate'] = pd.to_datetime(df['accidentdate'])
df

,node_id,accidentdate,accidenttime,accident_type,day_of_week,dca_code,light_condition,no_of_vehicles,no_persons,no_persons_inj_2,...,no_of_events,no_of_objs,object_type,lat,long,n_accidents_at_node,n_inj_at_node,n_deaths_at_node,casualties_at_node,fatal
0,2,2002-07-14,61500.0,1,1,113,1,2,3,1,...,1,0,0,-37.876840,145.477790,1.0,3.0,0.0,3.0,0.0
1,3,2002-12-30,48000.0,4,2,171,1,1,1,1,...,2,1,2,-37.680710,144.883700,1.0,1.0,0.0,1.0,0.0
2,4,2009-08-07,60300.0,1,6,110,1,3,4,0,...,2,0,0,-37.755268,144.970566,3.0,4.0,0.0,4.0,0.0
3,5,2001-11-29,57600.0,1,5,162,1,3,1,1,...,3,0,0,-37.692430,144.987160,1.0,1.0,0.0,1.0,0.0
4,6,2000-03-22,48600.0,1,4,135,1,2,2,0,...,2,0,0,-37.784336,144.969735,17.0,19.0,0.0,19.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192792,666055,2020-06-12,58200.0,1,5,130,1,2,2,0,...,1,0,0,-37.911115,145.236231,1.0,1.0,0.0,1.0,0.0
192793,666090,2020-06-12,54360.0,1,5,171,1,2,2,0,...,3,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192794,666093,2020-07-12,42600.0,6,0,182,1,1,3,0,...,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192795,666095,2020-10-16,57660.0,1,5,160,1,2,1,1,...,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

def tree_func(df, test_size, target_column):
    
    print('decision tree')
    print('target: ' + str(target_column))
    
    y =df[target_column].values
    X = df.drop(columns = [target_column])
    
    X_train, X_test, y_train, y_test = train_test_split (
        X, 
        y, 
        test_size=test_size, 
        random_state=1)

    clf = DecisionTreeClassifier(
        criterion = 'gini',
        max_depth = None,
        presort = False,
        max_features = None,
        random_state=1)

    clf.fit(X_train,y_train)
    prediction = clf.predict(X_test)

    print('results:')    
    print(sklearn.metrics.classification_report(y_test, prediction))
     
    c_m = confusion_matrix(y_test, prediction)
    print('confusion matrix')
    print(c_m)
  

In [25]:
dfb = df.dropna(axis=0, how='any')
dfb[['future_accidents']] = (dfb[['n_accidents_at_node']] > 1) * 1
dfb[['future_deaths']] = (( dfb['n_deaths_at_node'] -  dfb['no_persons_killed'] ) > 0) * 1
dfb[['future_casualties']] = (( dfb['casualties_at_node'] -  dfb['casualties'] ) > 0) * 1


dfb = dfb[[
      #'node_id', 
     # 'accidentdate',
      'accidenttime', 
      'accident_type',
      'day_of_week',
  #    'dca_code',
      'light_condition',
      'no_of_vehicles',
      'no_persons',
      'no_persons_inj_2',
      'no_persons_inj_3',
      'no_persons_killed',
      'police_attend',
      'road_geometry',
      'severity',
  #    'n_injured_t',
  #    'casualties',
 #     'no_of_events',
      'no_of_objs',
      'object_type',
      'lat',
      'long',
     # 'n_accidents_at_node',
     # 'n_inj_at_node',
     # 'n_deaths_at_node',
     # 'casualties_at_node',
     # 'fatal' ,
      'future_casualties']]



tree_func(dfb, .2, 'future_casualties')

decision tree
target: future_casualties
results:
              precision    recall  f1-score   support

           0       0.92      0.91      0.91     33746
           1       0.40      0.42      0.41      4774

   micro avg       0.85      0.85      0.85     38520
   macro avg       0.66      0.67      0.66     38520
weighted avg       0.85      0.85      0.85     38520

confusion matrix
[[30707  3039]
 [ 2754  2020]]
